In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [35]:
import sys
sys.path.append('../src')
from data.audio_dataset import AudioSpectrogramDataset
from models.Old_models.Generator import Generator
from models.Old_models.Discriminator import Discriminator
from models.PGAN_model.PGenerator import PGenerator
from models.PGAN_model.PDiscriminator import PDiscriminator

import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import librosa
import soundfile as sf
from torch.utils.data import DataLoader

In [36]:
# # hyperparameters
# z_dim = 100
batch_size = 32
num_epochs = 50
latent_dim = 100
# learning_rate = 0.0002
# beta1 = 0.5

# base_directory = "../data/raw/NSynth/audio"
base_directory = "../data/raw/Bass"

dataset = AudioSpectrogramDataset(base_directory=base_directory, spectro_type='mel')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [37]:
first_batch = next(iter(dataloader))
print(first_batch.size())

torch.Size([32, 1, 256, 160])


In [38]:
temp = torch.rand((32, 1, 16, 10))
# temp = torch.unsqueeze(temp, dim=0)
print(temp.shape)

torch.Size([32, 1, 16, 10])


In [39]:
import torch.nn.functional as F
goal_size = [32, 1, 8, 5]

In [40]:
low_res_real = F.adaptive_avg_pool2d(temp, output_size=(8, 5))
low_res_real2 = F.interpolate(temp, size=goal_size[-2:], mode='nearest')

In [41]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    n_devices = torch.cuda.device_count()
else:
    device = torch.device("cpu")
    n_devices = 1    
device

device(type='cuda', index=0)

In [42]:
first_batch_low_resolution = F.adaptive_avg_pool2d(first_batch, output_size=(16, 10))
first_batch_low_resolution = first_batch_low_resolution.to(device)
first_batch_low_resolution.shape

torch.Size([32, 1, 16, 10])

In [43]:
print(f'low_res_real: {low_res_real.shape}')
print(f'low_res_real2: {low_res_real2.shape}')

low_res_real: torch.Size([32, 1, 8, 5])
low_res_real2: torch.Size([32, 1, 8, 5])


In [54]:
# t = torch.nn.Upsample(scale_factor=0.5, mode='nearest')
# print(t(temp).shape)

torch.Size([32, 1, 8, 10])


In [45]:
discriminator = PDiscriminator().to(device)
# print(next(discriminator.parameters()).device)
discriminator.add_next_block(new_depth=256)
# print(next(discriminator.parameters()).device)
# discriminator.add_next_block(new_depth=128)
discriminator.set_alpha(0.2)
# print(first_batch_low_resolution.device)
dummy_output = discriminator(first_batch_low_resolution)
# print(dummy_output.shape)
# print(dummy_output)

In [16]:
p_latent_dim = 100
dummy_latent_vector = torch.randn((1, p_latent_dim))
generator = PGenerator()
generator.add_next_block(new_depth=256)
generator.add_next_block(new_depth=128)
generator.add_next_block(new_depth=64)
generator.add_next_block(new_depth=32)
generator.set_alpha(0.2)
dummy_output = generator(dummy_latent_vector)
print(dummy_output.shape)

output_size: (8, 5)
output_size: (16, 10)
output_size: (32, 20)
output_size: (64, 40)
output_size: (128, 80)
torch.Size([1, 1, 128, 80])


In [36]:
print(generator)

PGenerator2(
  (blocks): ModuleList(
    (0): ModuleList(
      (0): Sequential(
        (0): Upsample(scale_factor=2.0, mode=nearest)
        (1): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): LeakyReLU(negative_slope=0.2)
        (3): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
        (1): Upsample(scale_factor=2.0, mode=nearest)
      )
    )
  )
  (normalizationLayer): NormalizationLayer()
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=10240, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (base_block): ModuleList(
    (0): Sequential(
      (0): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1,

In [7]:
dummy_latent_vector = torch.randn((1, latent_dim))
generator = Generator3(height=256, width=172, transformation_type='mel', latent_dim=latent_dim)
dummy_output = generator(dummy_latent_vector)
print(dummy_output.shape)

torch.Size([1, 256, 16, 10])
torch.Size([1, 128, 33, 21])
torch.Size([1, 64, 67, 43])
torch.Size([1, 32, 134, 86])
torch.Size([1, 16, 268, 172])
torch.Size([1, 1, 256, 173])
torch.Size([1, 1, 256, 173])


In [16]:
discriminator = Discriminator(height=256, width=173, transformation_type='mel')
print(f'dummy shape: {dummy_output.shape}')
discriminator_output = discriminator(dummy_output)
print(discriminator_output.shape)

dummy shape: torch.Size([1, 1, 256, 173])
torch.Size([1, 16, 268, 172])
torch.Size([1, 32, 134, 86])
torch.Size([1, 64, 67, 43])
torch.Size([1, 128, 33, 21])
torch.Size([1, 256, 16, 10])
torch.Size([1, 40960])
torch.Size([1, 1])
torch.Size([1, 1])
